In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from ipywidgets import interact
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')

In [ ]:
fully_merged = pd.read_csv('../data/fully_merged_to_be_imported.csv')
fully_merged.describe(exclude = ['object', 'int64', 'bool'])

In [ ]:
sample_years = fully_merged.loc[fully_merged['Year'].isin([2018, 2010, 2000])]

fig = px.histogram(sample_years, 
                   x = "% Population Living on <$5.50/Day", 
                   color = 'Continent', 
                   barmode = 'relative', 
                   nbins = 20)
fig.show()

In [ ]:
fig = px.histogram(sample_years, 
                   x = "% Population Living on <$1.90/Day",
                   color = 'Year',
                   color_discrete_sequence = px.colors.qualitative.Dark2,
                   marginal = 'box')
fig.show()

In [ ]:
fig = px.histogram(fully_merged, x = "% Income Held by Top 10%", pattern_shape = "Continent")
fig.show()

In [ ]:
fig = px.scatter(fully_merged, 
                 x = "% Income Held by Top 10%", 
                 y = "% Population Living on <$1.90/Day", 
                 color = "Continent",
                 size= "GDP_Per_Capita", 
                 hover_data = ["Country", "Year"])
fig.show()

In [ ]:
fig = px.scatter(fully_merged, 
                 x = "% Income Held by Top 10%", 
                 y = "% Population Living on <$5.50/Day", 
                 color = "Continent",
                 size= "GDP_Per_Capita", 
                 hover_data = ["Country", "Year"])
fig.show()

Deep poverty is less correlated with income inequality, but marginally less deep poverty is suspect in this regard--particularly for countries in North America, South America and Africa. This conclusion is less true for countries in Europe and Asia. The foregoing suggests that income distribution among percentiles is to some extent, a zero-sum game.

In [ ]:
fig = px.scatter(fully_merged, 
                 x = "% Income Held by Top 10%", 
                 y = "GDP_Per_Capita", 
                 hover_data = ["Country", "Year"], 
                 trendline = "ols")
fig.show()

Contrary to the above conclusions, plotting income inequality against GDP per capita indicates a negative correlation between the two. To the extent that more income distribution is not a zero-sum game, increased concentration of income at the top end of the distribution does not result in higher productivity. Note the distinction between wealth and income for clarity.

In [ ]:
relevant_columns = ["GDP_Per_Capita", 
                    "% Income Held by Top 10%", 
                    "% Population Living on <$5.50/Day", 
                    "% Population Living on <$1.90/Day", 
                    "US Sanctions"]
corrs = fully_merged[relevant_columns].corr()

figure = ff.create_annotated_heatmap(
    z = corrs.values,
    x = list(corrs.columns),
    y = list(corrs.index),
    annotation_text = corrs.round(2).values,
    showscale = True)
figure.show()

In [ ]:
list_of_countries = list(fully_merged["Country"].sample(n = 10))
countries_series = fully_merged["Country"].loc[fully_merged["Country"].isin(list_of_countries)]
relevant_columns = ["GDP_Per_Capita", 
                    "% Income Held by Top 10%", 
                    "% Population Living on <$5.50/Day"]
@interact(y = relevant_columns)
def make_boxen(y):
    sns.set(rc={'figure.figsize':(15,10)})
    sns.boxenplot(x = countries_series, y = y, data = fully_merged, orient = "v", palette = "Set3", k_depth = "full")
    plt.show()